In [105]:
#import desired libararies
import pandas as pd
import numpy as np
import nltk
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from bs4 import BeautifulSoup

In [106]:
#import data
#read data from directory
#distinguish data
#review and sentiment
#you can copy the path by copy path option
data=pd.read_csv('../data/interim/data_validation.csv')
df=data.copy()
#first 5 rows
df.head()

,review
0,I got this one a few weeks ago and love it! It...
1,kill every one
2,I donot like anyone
3,I love animals
4,wwe is crazy


In [107]:
df.shape

(6, 1)

In [108]:
df.columns

Index(['review'], dtype='object')

In [109]:
#there is no missing values
#summition format
df.isnull().sum()
#bool format
df.isnull().any()

review    False
dtype: bool

In [110]:
#note that two columns of df are numerical
df.describe().T

,count,unique,top,freq
review,6,6,I got this one a few weeks ago and love it! It...,1


In [111]:
for i in df.columns :
  print(f' for {i} the unique values are : ')
  print()
  print(df[i].value_counts())
  print()

 for review the unique values are : 

review
I got this one a few weeks ago and love it! It's modern, light but filled with true complexities of life. It questions and answers, just like other Eytan Fox movies. This is my favorite, along with Jossi & Jagger. This pictures a lot more, universally, than only the bubbles we may live in. You don't need to be Jewish or homosexual to enjoy this - I'm not, but the movie goes directly to my top ten movies. At first it seems like pure entertainment but it does make you think further. Relationships we have to live with are superficial, meaningful, deep, fatal, you name it. You don't know what's coming, and you definitely don't know where this story is heading as you watch it the first time. It is worth seeing several times. Fox movies include great bonus material - here a great music video and "the making of" (including explanation of the title, interviewing Lior Ashknenazi who plays himself in the movie and Arabs with doubts about the Israeli l

In [112]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   review  6 non-null      object
dtypes: object(1)
memory usage: 180.0+ bytes


In [113]:
#prepare stop words , tokens and html parsing
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [114]:
#parse html code and erase special pattern using beautiful soup and regex
#beautiful soup is not only for web scrapping but aslo for html parsing basicly
def noise_removal(text) :
  soup=BeautifulSoup(text,"html.parser")
  text=soup.get_text()
  #remove special charcters
  text=re.sub('\[[^]]*\]','',text)
  return text

In [115]:
#apply removal function on review column
df['review']=df['review'].apply(noise_removal)

In [116]:
#data after cleaning html parsers and speacial characters
df.head()

,review
0,I got this one a few weeks ago and love it! It...
1,kill every one
2,I donot like anyone
3,I love animals
4,wwe is crazy


In [117]:
#apply function to return stem every word for every instace
def stemmer(text):
  ps=nltk.porter.PorterStemmer()
  #convert list to text again using .join
  text=' '.join([ps.stem(word) for word in text.split()])
  return text

In [118]:
#apply stemmer function on review column
df['review']=df['review'].apply(stemmer)

In [119]:
#data after applying stemming
df.head()

,review
0,i got thi one a few week ago and love it! it' ...
1,kill everi one
2,i donot like anyon
3,i love anim
4,wwe is crazi


In [120]:
#removing stop words
stopwords

<WordListCorpusReader in 'C:\\Users\\hp\\AppData\\Roaming\\nltk_data\\corpora\\stopwords'>

In [121]:
#tokenization is important for removing stop words later as comparing them with list of stop words later
#create tokenizer object
token=ToktokTokenizer()
#stop words in english
stopwords=nltk.corpus.stopwords.words('english')
def remove_stopwords(text , is_lower_case = False) :
  #text maybe small or capital
  tokens=token.tokenize(text)
  #remove any white blank for every elemnt for list of words using list comprehensive
  tokens=[i.strip() for i in tokens]
  if is_lower_case :
    new_tokens = [i for i in  tokens if i not in stopwords]
  else :
    new_tokens = [i for i in tokens if i.lower() not in stopwords]
  #return filtered tokens converting list to string
  filtered_tokens=' '.join(new_tokens)
  return filtered_tokens

In [122]:
#apply remove stop words function on review column
df['review']=df['review'].apply(remove_stopwords)

In [123]:
#data after applying stop words removing
df.head()

,review
0,"got thi one week ago love ! ' modern , light f..."
1,kill everi one
2,donot like anyon
3,love anim
4,wwe crazi


In [130]:
import pickle
#use model
for i in range(len(df)):
    #TFIDF vtransform
    tf = pickle.load(open('model_tfidf.pkl', 'rb'))
    tf_df=tf.transform(df.iloc[i])
    #use model
    #prediction
    model = pickle.load(open('model_log.pkl', 'rb'))
    df_predict=model.predict(tf_df)
    print(f'for statement {i} : ',f'{data.iloc[i][0]}')
    print()
    print('it is : ',df_predict)
    print('-'*100)


for statement 0 :  I got this one a few weeks ago and love it! It's modern, light but filled with true complexities of life. It questions and answers, just like other Eytan Fox movies. This is my favorite, along with Jossi & Jagger. This pictures a lot more, universally, than only the bubbles we may live in. You don't need to be Jewish or homosexual to enjoy this - I'm not, but the movie goes directly to my top ten movies. At first it seems like pure entertainment but it does make you think further. Relationships we have to live with are superficial, meaningful, deep, fatal, you name it. You don't know what's coming, and you definitely don't know where this story is heading as you watch it the first time. It is worth seeing several times. Fox movies include great bonus material - here a great music video and "the making of" (including explanation of the title, interviewing Lior Ashknenazi who plays himself in the movie and Arabs with doubts about the Israeli life styles).

it is :  ['p

KeyboardInterrupt: 